Клонируем репозиторий ```EasyOCR```

In [1]:
!git clone https://github.com/4ervonec19/EasyOCR.git

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2759, done.
remote: Counting objects: 100% (855/855), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 2759 (delta 785), reused 767 (delta 762), pack-reused 1904 (from 1)
Receiving objects: 100% (2759/2759), 171.32 MiB | 29.02 MiB/s, done.
Resolving deltas: 100% (1694/1694), done.
Updating files: 100% (314/314), done.


Переходим

In [2]:
import os
os.chdir("EasyOCR")
os.chdir("trainer")

In [3]:
%%bash
ls

all_data
config_files
craft
dataset.py
model.py
modules
README.md
saved_models
test_.py
trainer.ipynb
train.py
utils.py


In [4]:
os.chdir("all_data")

In [5]:
%%bash
pwd

/kaggle/working/EasyOCR/trainer/all_data


Копируем данные

In [6]:
%%bash
cp -r /kaggle/input/images-for-easy-ocr/train/train /kaggle/working/EasyOCR/trainer/all_data

In [7]:
%%bash
cp -r /kaggle/input/images-for-easy-ocr/val/val /kaggle/working/EasyOCR/trainer/all_data

In [8]:
os.chdir("..")

In [9]:
%%bash
pwd

/kaggle/working/EasyOCR/trainer


Теперь нужно заменить в ```JSON``` на имена директории

In [10]:
import pandas as pd
input_csv_path = "/kaggle/working/EasyOCR/trainer/all_data/train/labels.csv"
df = pd.read_csv(input_csv_path)
def replace_path(path):
    filename = path.split('/')[-1]
    new_path = f"/kaggle/working/EasyOCR/trainer/all_data/train/{filename}"
    return new_path

df['filename'] = df['filename'].apply(replace_path)
df.to_csv(input_csv_path, index=False)
df['filename'].values[0]

'/kaggle/working/EasyOCR/trainer/all_data/train/0_17765.jpg'

In [11]:
input_csv_path = "/kaggle/working/EasyOCR/trainer/all_data/val/labels.csv"
df = pd.read_csv(input_csv_path)
def replace_path_val(path):
    filename = path.split('/')[-1]
    new_path = f"/kaggle/working/EasyOCR/trainer/all_data/val/{filename}"
    return new_path

df['filename'] = df['filename'].apply(replace_path_val)
df.to_csv(input_csv_path, index=False)
df['filename'].values[0]

'/kaggle/working/EasyOCR/trainer/all_data/val/0_lhr_0_861_01020023.jpg'

In [12]:
input_csv_path = "/kaggle/working/EasyOCR/trainer/all_data/train/labels.csv"
df = pd.read_csv(input_csv_path)
df.head()

,filename,words
0,/kaggle/working/EasyOCR/trainer/all_data/train...,ФОТОКОНКУРС
1,/kaggle/working/EasyOCR/trainer/all_data/train...,Теремок
2,/kaggle/working/EasyOCR/trainer/all_data/train...,стул
3,/kaggle/working/EasyOCR/trainer/all_data/train...,Нина
4,/kaggle/working/EasyOCR/trainer/all_data/train...,мчс россии


In [13]:
input_csv_path = "/kaggle/working/EasyOCR/trainer/all_data/val/labels.csv"
df = pd.read_csv(input_csv_path)
df.head()

,filename,words
0,/kaggle/working/EasyOCR/trainer/all_data/val/0...,Чуйкова
1,/kaggle/working/EasyOCR/trainer/all_data/val/3...,СОГЛАСОВАНО
2,/kaggle/working/EasyOCR/trainer/all_data/val/0...,Рымша Лезина истово
3,/kaggle/working/EasyOCR/trainer/all_data/val/0...,лохмы Казачек
4,/kaggle/working/EasyOCR/trainer/all_data/val/0...,ваш консалтинг ооо ваш консалтинг Бухгалтерско...


Теперь пробуем обучить

In [14]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [15]:
cudnn.benchmark = True
cudnn.deterministic = False

In [16]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [17]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/train
opt.select_data: ['all_data/train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/train	 dataset: all_data/train
all_data/train/
sub-directory:	/.	 num samples: 50794
num total samples of all_data/train: 50794 x 1.0 (total_data_usage_ratio) = 50794
num samples of all_data/train per batch: 64 x 1.0 (batch_ratio) = 64


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


--------------------------------------------------------------------------------
Total_batch_size: 64 = 64
--------------------------------------------------------------------------------
dataset_root:    all_data/val	 dataset: /
all_data/val/
sub-directory:	/.	 num samples: 12687
--------------------------------------------------------------------------------
No Transformation module specified
model input parameters 64 600 20 1 256 256 208 68 None VGG BiLSTM CTC


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/kaggle/working/EasyOCR/trainer/train.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. A

loading pretrained model from saved_models/best_accuracy.pth
Model:
DataParallel(
  (module): Model(
    (FeatureExtraction): VGG_FeatureExtractor(
      (ConvNet): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): ReLU(inplace=True)
        (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (9): ReLU(inplace=True)
        (10): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
        (11): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        

/kaggle/working/EasyOCR/trainer/train.py:177: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


training time:  309.47163367271423
[2000/30000] Train loss: 0.96759, Valid loss: 1.42256, Elapsed_time: 309.47219
Current_accuracy : 38.725, Current_norm_ED  : 0.6648
Best_accuracy    : 38.725, Best_norm_ED     : 0.6648
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
Наумкина                  | Наумкина                  | 0.4886	True
гробик Молдавия           | гробик аррав              | 0.0000	False
--------------------------------------------------------------------------------
validation time:  81.42475771903992
training time:  301.41205883026123
[4000/30000] Train loss: 0.90867, Valid loss: 1.42504, Elapsed_time: 692.30958
Current_accuracy : 39.198, Current_norm_ED  : 0.6655
Best_accuracy    : 39.198, Best_norm_ED     : 0.6655
-----------------------------------------------------------------

SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
